In [1]:
name = '2017-01-20-function-quirks'
title = 'Some peculiarities of using functions in Python'
tags = 'basics'
author = 'Denis Sergeev'

In [2]:
from nb_tools import connect_notebook_to_post
from IPython.core.display import HTML, Image

html = connect_notebook_to_post(name, title, tags, author)

# Some peculiarities of using functions in Python

## Resources
* [Write Pythonic Code Like a Seasoned Developer Course Demo Code](https://github.com/mikeckennedy/write-pythonic-code-demos)
* [Scipy Lecture notes](http://www.scipy-lectures.org/intro/language/functions.html)

In [3]:
HTML(html)